### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [4]:
# read the data into pandas dataframes
BY_T24_GF_302a = pd.read_csv("lca/cleaned/302_BY_T24_GF_lca.csv")

# delete any Metazoa hits because they're trypsin
BY_T24_GF_302b = BY_T24_GF_302a[BY_T24_GF_302a.kingdom != 'Metazoa']

# keep only entries to the phylum level
BY_T24_GF_302 = BY_T24_GF_302b[BY_T24_GF_302b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(BY_T24_GF_302))

BY_T24_GF_302.head(6)

# of phylum peptides =  21


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
3,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,MHLMHLTK,Dorea,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,...,Dorea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,CQMYMK,Anaerostipes rhamnosivorans,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,...,Anaerostipes,NaN,NaN,NaN,Anaerostipes rhamnosivorans,NaN,NaN,NaN,NaN,NaN
114,YLWMSVK,Actinomycetia,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,TAHGPSK,Streptomyces,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,...,Streptomyces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [7]:
BY_T24_GF_302_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/302A_BY_T24_GF_DN50_ILnaafs.csv")
BY_T24_GF_302_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/302B_BY_T24_GF_DN50_ILnaafs.csv")
BY_T24_GF_302_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/302C_BY_T24_GF_DN50_ILnaafs.csv")


frames = [BY_T24_GF_302_NAAFa, BY_T24_GF_302_NAAFb]

BY_T24_GF_302_NAAF = pd.concat(frames, sort=False)

BY_T24_GF_302_NAAF.set_index('stripped_peptide')
BY_T24_GF_302_NAAF = BY_T24_GF_302_NAAF.loc[:, ~BY_T24_GF_302_NAAF.columns.str.contains('^Unnamed')]

BY_T24_GF_302_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(BY_T24_GF_302_NAAF))

print('column names:', BY_T24_GF_302_NAAF.columns)

BY_T24_GF_302_NAAF.head()

# of total peptides =  323
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,LSSPATLNSR,42900000.0,4290000.0
1,LSSPATLNSR,42900000.0,4290000.0
2,LNSPATLNSR,383000.0,38300.0
3,VVTVSLPR,58200000.0,7275000.0
4,VATVSPLR,79800000.0,9975000.0


In [8]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(BY_T24_GF_302["peptide"], BY_T24_GF_302_NAAF["peptide"])

print(over)

['CQMYMK' 'LATVLSPR' 'VATVSPLR' 'YLWMSVK']


In [9]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

BY_T24_GF_302.set_index('peptide', inplace=True)
BY_T24_GF_302_NAAF.set_index('peptide', inplace=True)

BY_T24_GF_302_Phy = BY_T24_GF_302.join(BY_T24_GF_302_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(BY_T24_GF_302_Phy))

BY_T24_GF_302_Phy.head()

# of total phylum-level peptides =  32


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
CQMYMK,Anaerostipes rhamnosivorans,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,NaN,...,NaN,NaN,Anaerostipes rhamnosivorans,NaN,NaN,NaN,NaN,NaN,72000.0,12000.0
DPNNPAQK,Candidatus Aminicenantes,Bacteria,NaN,NaN,NaN,Candidatus Aminicenantes,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FAVPAEDK,Variovorax,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Betaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GDHNYLK,Pasteurellaceae,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# write to a csv

BY_T24_GF_302_Phy.to_csv("lca/NAAF/BY_T24_GF_302_Phy_naaf.csv")